In [65]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import imghdr
import pandas as pd
from PIL import Image

### Set up device

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [17]:
data_dir = '../data'
train_dir = 'train_data'
test_dir = 'test_data_v2'

In [18]:
image_exts = ['jpg', 'jpeg', 'bmp', 'png']

### check dodgy data

In [35]:
# check train dataset
for class_name in ['AI', 'human']:
    for image in os.listdir(os.path.join(data_dir, train_dir, class_name)):
        image_path = os.path.join(data_dir, train_dir, class_name, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not have a valid extension')
        except Exception as e:
            print('Issue with image {}'.format(image_path))


KeyboardInterrupt: 

In [10]:
# check test dataset
for image in os.listdir(os.path.join(data_dir, test_dir)):
    image_path = os.path.join(data_dir, test_dir, image)
    try:
        img = cv2.imread(image_path)
        tip = imghdr.what(image_path)
        if tip not in image_exts:
            print('Image not have a valid extension')
    except Exception as e:
        print('Issue with image {}'.format(image_path))

### load data

In [19]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [53]:
dataset = datasets.ImageFolder(root=os.path.join(data_dir, train_dir), transform=transform)

In [21]:
train_size = int(len(dataset) * 0.8)
val_size = int(len(dataset) - train_size)
(train_size, val_size)

(63960, 15990)

In [22]:
train_data, val_data = random_split(dataset, [train_size, val_size])

In [41]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True, num_workers=4)

### Build CNN model

In [42]:
import torch.nn as nn
import torch.nn.functional as F

In [43]:
class BinaryCNN(nn.Module):
    def __init__(self):
        super(BinaryCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64* 32 *32, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [44]:
model = BinaryCNN().to(device)

In [45]:
model.parameters

<bound method Module.parameters of BinaryCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)>

### Train the model

In [46]:
import torch.optim


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lr=0.0001, params=model.parameters())

num_epoches = 20
for epoch in range(num_epoches):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device, dtype=torch.float32)
        
        labels = labels.view(-1, 1)
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: {epoch + 1}, Loss: {running_loss/len(train_loader)}")

Epoch: 1, Loss: 0.326528696089983
Epoch: 2, Loss: 0.25059890758991243
Epoch: 3, Loss: 0.22983109006285668
Epoch: 4, Loss: 0.21278776063024998
Epoch: 5, Loss: 0.1952558891773224
Epoch: 6, Loss: 0.18242621363699435
Epoch: 7, Loss: 0.16301690843701364
Epoch: 8, Loss: 0.1466332373768091
Epoch: 9, Loss: 0.13292466098070144
Epoch: 10, Loss: 0.11903331771492959
Epoch: 11, Loss: 0.10411828487738967
Epoch: 12, Loss: 0.09500750358402729
Epoch: 13, Loss: 0.08467168880999089
Epoch: 14, Loss: 0.07340789398923517
Epoch: 15, Loss: 0.06622624344006181
Epoch: 16, Loss: 0.05955776847153902
Epoch: 17, Loss: 0.05251406927779317
Epoch: 18, Loss: 0.04673792872391641
Epoch: 19, Loss: 0.043400810251012445
Epoch: 20, Loss: 0.035397361448965964


### Evaluate|

In [49]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.view(-1, 1).float()
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 96.60%


### Predict

In [58]:
test_df = pd.read_csv('../data/test.csv')

In [88]:
image_list = list(test_df['id'])

In [111]:
outputs_list = []
for image_path in image_list:
    image = Image.open(os.path.join(data_dir, image_path)).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        logit = model(image)
    prob = torch.sigmoid(logit)
    prediction = (prob>0.5).float()
    outputs_list.append(int(prediction.item()))

In [113]:
test_df['label'] = outputs_list

In [114]:
test_df.to_csv('../data/result/CNN_pred.csv', index=False)